In [1]:
import fcl 
import math 
import numpy as np 
from tf.transformations import quaternion_from_euler 

print("Packages Loaded.")

Packages Loaded.


## Tutorial: Flexible Collision Library 

In [2]:
def make_box(name, pos, rot, size): 
    return {"name":name, "type":"box", "info":pos+rot+size}

""" Flexible Collision Library """
class PyFCL:
    def __init__(self, _num_max_contacts = 100, _verbose = True):
        self.geom_lst = []
        self.objs_lst = []
        self.name_lst = [] 
        self.verbose  = _verbose
        self.req      = fcl.CollisionRequest()
        self.res      = fcl.CollisionResult()
        self.cdata    = fcl.CollisionData(request = self.req)

    # capsule type: [position[0], position[1], position[2], rpy[0], rpy[1], rpy[2], height, radius]
    # box type: [position[0], position[1], position[2], rpy[0], rpy[1], rpy[2], size[0], size[1], size[2]]
    def fclize_obj(self, _name, _type, obj_info): 
        obj_name, obj_type = str(_name), str(_type) 
        if obj_type == "capsule":
            obj_pos, obj_rpy, obj_height, obj_radius = obj_info[:3], obj_info[3:6], obj_info[6], obj_info[7]
            obj_q = quaternion_from_euler (obj_rpy[0], obj_rpy[1], obj_rpy[2]) 
            obj_t = np.array(obj_pos)
            obj_g = fcl.Capsule(obj_radius, obj_height)
            obj_t = fcl.Transform(obj_q, obj_t)
            obj_o = fcl.CollisionObject(obj_g, obj_t)
 
        if obj_type == "box": 
            obj_pos, obj_rpy, obj_size = obj_info[:3], obj_info[3:6], obj_info[6:] 
            obj_q = quaternion_from_euler (obj_rpy[0], obj_rpy[1], obj_rpy[2]) 
            obj_t = np.array(obj_pos)            
            obj_g = fcl.Box(obj_size[0], obj_size[1], obj_size[2])
            obj_t = fcl.Transform(obj_q, obj_t)
            obj_o = fcl.CollisionObject(obj_g, obj_t)
        self.geom_lst.append(obj_g)
        self.objs_lst.append(obj_o)
        self.name_lst.append(obj_name)
        return obj_o 

    def one2one_cc(self, fclize_obj1, fclize_obj2):
        collision = fcl.collide(fclize_obj1, fclize_obj2, self.req, self.res)
        """ Collision Check """
        if collision: 
            if self.verbose: 
                print('*WARNING* {} in collision with object {}!'.format(self.name_lst[0], self.name_lst[1]))
            return True # Collision detect
        else: 
            if self.verbose: 
                print("Collision Free")
            return False  # Collision free 

    def one2many_cc(self, obj, obj_list):
        manager = fcl.DynamicAABBTreeCollisionManager()
        manager.registerObjects(obj_list)
        manager.setup()
        manager.collide(obj, self.cdata, fcl.defaultCollisionCallback)
        collision = self.cdata.result.is_collision
        if self.verbose: 
            objs_in_collision = set() 
            # Create map from geometry IDs to objects
            geom_id_to_obj  = {id(geom) : obj for geom, obj in zip(self.geom_lst, self.objs_lst)}
            # Create map from geometry IDs to string names
            geom_id_to_name = {id(geom) : name for geom, name in zip(self.geom_lst, self.name_lst)}
            for contact in self.cdata.result.contacts:
                coll_geom_0 = contact.o1
                coll_geom_1 = contact.o2

                # Get their names
                coll_names = [geom_id_to_name[id(coll_geom_0)], geom_id_to_name[id(coll_geom_1)]]
                coll_names = tuple(sorted(coll_names))
                objs_in_collision.add(coll_names)

            for coll_pair in objs_in_collision:
                pass
                print('*WARNING* {} in collision with object {}!'.format(coll_pair[0], coll_pair[1]))
        """ Collision Check """
        if collision: 
            return True # Collision detect
        else: 
            return False  # Collision free 


    def one2one_collision_check(self, obj_info, link_list, type):
        """ Object part """
        obj_type       = obj_info[0]
        obj_name       = obj_info[1]
        obj_value = [float(param) for param in obj_info[2:]]
        obj_position   = np.array([obj_value[0], obj_value[1], obj_value[2]])
        qx_obj, qy_obj, qz_obj, qw_obj= quaternion_from_euler (obj_value[3], obj_value[4], obj_value[5])
        obj_quaternion = np.array([qx_obj, qy_obj, qz_obj, qw_obj])

        if obj_type == "box": 
            obj_gb = fcl.Box(obj_value[6], obj_value[7], obj_value[8])
            obj_tb = fcl.Transform(obj_quaternion, obj_position)
            obj_ob = fcl.CollisionObject(obj_gb, obj_tb)
        self.geom_lst.append(obj_gb)
        self.objs_lst.append(obj_ob)
        self.name_lst.append(obj_name)

        """ Robot link part """
        link_ocap_lst = [] 
        if type == "capsule":
            for idx, link in enumerate(link_list): 
                link_name     = str(link[0])
                link_value    = [float(param) for param in link[1:]]
                link_position = np.array([link_value[0], link_value[1], link_value[2]])
                qx_link, qy_link, qz_link, qw_link = quaternion_from_euler (link_value[3], link_value[4], link_value[5])
                link_quaternion = np.array([qx_link, qy_link, qz_link, qw_link])  
                link_height, link_rad = link_value[6], link_value[7]
                link_gcap = fcl.Capsule(link_rad, link_height)
                link_tcap = fcl.Transform(link_quaternion, link_position)
                link_ocap = fcl.CollisionObject(link_gcap, link_tcap)
                self.geom_lst.append(link_gcap)
                self.objs_lst.append(link_ocap)
                self.name_lst.append(link_name)
                if link_height ==0 or link_rad ==0:
                    pass
                else:
                    link_ocap_lst.append(link_ocap)
                
        if type == "box": 
            for idx, link in enumerate(link_list): 
                link_name     = str(link[0])
                link_value    = [float(param) for param in link[1:]]
                link_position = np.array([link_value[0], link_value[1], link_value[2]])
                qx_link, qy_link, qz_link, qw_link = quaternion_from_euler (link_value[3], link_value[4], link_value[5])
                link_quaternion = np.array([qx_link, qy_link, qz_link, qw_link])  
                link_x, link_y, link_z = link_value[6], link_value[7], link_value[8]
                link_gcap = fcl.Box(link_x, link_y, link_z)
                link_tcap = fcl.Transform(link_quaternion, link_position)
                link_ocap = fcl.CollisionObject(link_gcap, link_tcap)
                self.geom_lst.append(link_gcap)
                self.objs_lst.append(link_ocap)
                self.name_lst.append(link_name)
                link_ocap_lst.append(link_ocap)   

        in_collision = np.ones(len(link_ocap))
        """ Collision Check """
        collision = fcl.collide(obj_ob, link_ocap, self.req, self.res)
        if collision == 0:
            in_collision[idx] = 0

        if sum(in_collision) == 0:
            return False
        else: 
            return True

In [3]:
""" Sample """
objinfo1 = [1, 2, 3, 0, 0, 0, 1, 1]
type1 = 'capsule'
name1 = 'obs1'

objinfo2 = [1, 2, 3, 0, 0, 0, 1, 1, 1]
type2 = 'box'
name2 = 'obs2'

objinfo3 = [1, 2, 3, 0, 0, 0, 1, 1, 1]
type3 = 'box'
name3 = 'obs3'

objinfo4 = [1, 2, 3, 0, 0, 0, 1, 1, 1]
type4 = 'box'
name4 = 'obs4'

objinfo5 = [5, 5, 5, 0, 0, 0, 1, 1, 1]
type5 = 'box'
name5 = 'obs5'


### One to Many Collision Check

In [4]:
""" Object Instance """
fclize = PyFCL() 

a = fclize.fclize_obj(name1, type1, objinfo1)
b = fclize.fclize_obj(name2, type2, objinfo2)
obj_list = [a,b]

""" Check Collision """
one2many_result = fclize.one2many_cc(a , obj_list)
print("Is collision? %s"%one2many_result)


*WARNING* obs1 in collision with object obs2!
*WARNING* obs1 in collision with object obs1!
Is collision? True


### One to One Collision Check 

In [5]:
""" Object Instance """
fclize = PyFCL() 
c = fclize.fclize_obj(name3, type3, objinfo3)
d = fclize.fclize_obj(name4, type4, objinfo4)
e = fclize.fclize_obj(name5, type5, objinfo5)

""" Check Collision """
one2one_result = fclize.one2one_cc(c, d)
print("Is collision? %s"%one2one_result)

one2one_result2 = fclize.one2one_cc(c, e)
print("Is collision? %s"%one2one_result2)

*WARNING* obs3 in collision with object obs4!
Is collision? True
Collision Free
Is collision? False
